# Homework 4

In [102]:
import pandas as pd
import sqlite3 as sql

## Joining Data

We want to analyse the number of cellphones per capita and to do so we need data on population sizes for countries in the years 1960-2020. This data can be loaded in through a csv-file named pop_data.csv.

In [103]:
pop_df = pd.read_csv('pop_data.csv')
pop_df

,Unnamed: 0,iso-3,1960,1961,1962,1963,1964,1965,1966,1967,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,ABW,54608.0,55811.0,56682.0,57475.0,58178.0,58782.0,59291.0,59522.0,...,101288.0,102112.0,102880.0,103594.0,104257.0,104874.0,105439.0,105962.0,106442.0,106585.0
1,1,AFE,130692579.0,134169237.0,137835590.0,141630546.0,145605995.0,149742351.0,153955516.0,158313235.0,...,537792950.0,552530654.0,567892149.0,583651101.0,600008424.0,616377605.0,632746570.0,649757148.0,667242986.0,685112979.0
2,2,AFG,8622466.0,8790140.0,8969047.0,9157465.0,9355514.0,9565147.0,9783147.0,10010030.0,...,29249157.0,30466479.0,31541209.0,32716210.0,33753499.0,34636207.0,35643418.0,36686784.0,37769499.0,38972230.0
3,3,AFW,97256290.0,99314028.0,101445032.0,103667517.0,105959979.0,108336203.0,110798486.0,113319950.0,...,366489204.0,376797999.0,387204553.0,397855507.0,408690375.0,419778384.0,431138704.0,442646825.0,454306063.0,466189102.0
4,4,AGO,5357195.0,5441333.0,5521400.0,5599827.0,5673199.0,5736582.0,5787044.0,5827503.0,...,24259111.0,25188292.0,26147002.0,27128337.0,28127721.0,29154746.0,30208628.0,31273533.0,32353588.0,33428486.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,261,XKX,947000.0,966000.0,994000.0,1022000.0,1050000.0,1078000.0,1106000.0,1135000.0,...,1791000.0,1807106.0,1818117.0,1812771.0,1788196.0,1777557.0,1791003.0,1797085.0,1788878.0,1790133.0
262,262,YEM,5542459.0,5646668.0,5753386.0,5860197.0,5973803.0,6097298.0,6228430.0,6368014.0,...,25475610.0,26223391.0,26984002.0,27753304.0,28516545.0,29274002.0,30034389.0,30790513.0,31546691.0,32284046.0
263,263,ZAF,16520441.0,16989464.0,17503133.0,18042215.0,18603097.0,19187194.0,19789771.0,20410677.0,...,52443325.0,53145033.0,53873616.0,54729551.0,55876504.0,56422274.0,56641209.0,57339635.0,58087055.0,58801927.0
264,264,ZMB,3119430.0,3219451.0,3323427.0,3431381.0,3542764.0,3658024.0,3777680.0,3901288.0,...,14265814.0,14744658.0,15234976.0,15737793.0,16248230.0,16767761.0,17298054.0,17835893.0,18380477.0,18927715.0


We can look at which rows contain NaN so we know where to fill in values.

In [104]:
pop_nan_rows = pop_df[pop_df.isna().any(axis=1)]
pop_nan_rows

,Unnamed: 0,iso-3,1960,1961,1962,1963,1964,1965,1966,1967,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
110,110,INX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,196,PSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3882986.0,3979998.0,4076708.0,4173398.0,4270092.0,4367088.0,4454805.0,4569087.0,4685306.0,4803269.0


We can check if the row with index 110 only contains NaN values.

In [119]:
pd.isnull(pop_nan_rows.loc[110]).all()

False

Since the row with index 110 only contains NaN values we can not use forward- or backward fill directly. Instead we can fill the row succesively with the average values for each of the columns.

In [105]:
for year in range(1960, 2021):
    mean_year = pop_df[str(year)].mean()
    pop_df.loc[110, str(year)] = mean_year

pop_df.loc[110]

Unnamed: 0                 110
iso-3                      INX
1960          117271174.712121
1961          118880721.420455
1962          121051087.992424
                    ...       
2016          304405110.230189
2017          308257495.324528
2018          312027628.377358
2019          315711047.777358
2020          319293645.637736
Name: 110, Length: 63, dtype: object

For the row with index 196 we can use backward fill since we have values in the later parts of the row.

In [106]:
pop_filled_df = pop_df.bfill(axis=1)
pop_filled_df.loc[196]

Unnamed: 0          196
iso-3               PSE
1960          1978248.0
1961          1978248.0
1962          1978248.0
                ...    
2016          4367088.0
2017          4454805.0
2018          4569087.0
2019          4685306.0
2020          4803269.0
Name: 196, Length: 63, dtype: object

In [107]:
pop_filled_df.isnull().values.any()

False

In [ ]:
pop_filled_df.loc[110]